<a href="https://colab.research.google.com/github/s-grzhang/gwc-chatbot/blob/main/gwc_bot_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import pandas as pd
from google.colab import drive
from datasets import Dataset

# Load the DataFrame from the CSV file
file_path = '/content/drive/My Drive/gwc_chatbot_data_revised.csv'
df = pd.read_csv(file_path)

# Combine User Input and Bot Response into a single text format suitable for training
df['text'] = df['User Input'] + " " + df['Bot Response']

# Create a Dataset from the DataFrame
dataset = Dataset.from_pandas(df[['text']])

# Display the dataset
print(dataset)


Dataset({
    features: ['text'],
    num_rows: 100
})


In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Split the dataset into train and test sets
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2)

# Define data collator
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.885700,2.824722
2,2.109500,2.566505
3,1.498900,2.443925
4,1.133800,2.452130
5,0.892300,2.451952
6,0.935600,2.549037
7,0.720500,2.622933
8,0.652100,2.650458
9,0.457500,2.668611
10,0.524100,2.685060


TrainOutput(global_step=400, training_loss=1.230842990875244, metrics={'train_runtime': 55.2967, 'train_samples_per_second': 14.467, 'train_steps_per_second': 7.234, 'total_flos': 52258406400000.0, 'train_loss': 1.230842990875244, 'epoch': 10.0})

In [ ]:
import torch

# Ensure the model is on the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_response(user_input):
    # Tokenize the input
    inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

    # Generate a response from the model
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,  # Ensure attention_mask is used
        max_length=128,  # Set the maximum length of the generated response
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,  # Use EOS token ID to handle padding
        eos_token_id=tokenizer.eos_token_id   # Ensure EOS token ID is set
    )
    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

# Test the chatbot
user_input = "What is the purpose of Girls Who Code?"
response = get_response(user_input)
print(response)

What is the purpose of Girls Who Code? Girls Who Code is to empower women through coding education, job opportunities, and community support. It is the only organization that offers coding programs and clubs specifically for girls. Girls Who Code aims to empower girls through teamwork, confidence building, and reducing feelings of isolation in a male-dominated field. It is the only organization that offers internship and scholarship opportunities specifically for girls. Girls Who Code is a nonprofit organization that offers coding programs and clubs specifically for girls. It is the only organization that offers internship and scholarship opportunities specifically for girls. Girls Who Code is a nonprofit organization that offers internship and scholarship opportunities specifically


In [ ]:
model.save_pretrained('/content/drive/My Drive/gwc_chatbot_model')
tokenizer.save_pretrained('/content/drive/My Drive/gwc_chatbot_tokenizer')


('/content/drive/My Drive/gwc_chatbot_tokenizer/tokenizer_config.json',
 '/content/drive/My Drive/gwc_chatbot_tokenizer/special_tokens_map.json',
 '/content/drive/My Drive/gwc_chatbot_tokenizer/vocab.json',
 '/content/drive/My Drive/gwc_chatbot_tokenizer/merges.txt',
 '/content/drive/My Drive/gwc_chatbot_tokenizer/added_tokens.json',
 '/content/drive/My Drive/gwc_chatbot_tokenizer/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel

# Correct path to your saved model and tokenizer
model_name = '/content/drive/My Drive/gwc_chatbot_model'
tokenizer_name = '/content/drive/My Drive/gwc_chatbot_tokenizer'

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Test the chatbot
def get_response(user_input):
    inputs = tokenizer.encode(user_input, return_tensors="pt")
    outputs = model.generate(inputs, max_length=45, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

user_input = "What is the purpose of Girls Who Code?"
response = get_response(user_input)
print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is the purpose of Girls Who Code? Girls Who Code (GWC) is to support women in computer science by fostering a supportive environment where women can work together to increase their skills and increase their visibility in the industry.
